# Import dependencies and instantiate model and import

In [1]:
import torch
import noisynn as nnn 
import timm 

model = timm.create_model('convnext_base.fb_in22k_ft_in1k', pretrained=False)
inp = torch.rand(4,3,224,224)

c:\Users\MinhNH\AppData\Local\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


To check the name of the layer you want to add NoisyNN to, use model.named_modules() to get the name of the layer you want to add NoisyNN to.

In this tutorial, I choose the layers to be `layers = ['stages.3.blocks.2','stages.2.blocks.26']` of the ConvNeXt-B model from `timm`.

In [2]:
print(dict(model.named_modules()).keys())

layers = ['stages.3.blocks.2','stages.2.blocks.26']

dict_keys(['', 'stem', 'stem.0', 'stem.1', 'stages', 'stages.0', 'stages.0.downsample', 'stages.0.blocks', 'stages.0.blocks.0', 'stages.0.blocks.0.conv_dw', 'stages.0.blocks.0.norm', 'stages.0.blocks.0.mlp', 'stages.0.blocks.0.mlp.fc1', 'stages.0.blocks.0.mlp.act', 'stages.0.blocks.0.mlp.drop1', 'stages.0.blocks.0.mlp.norm', 'stages.0.blocks.0.mlp.fc2', 'stages.0.blocks.0.mlp.drop2', 'stages.0.blocks.0.shortcut', 'stages.0.blocks.0.drop_path', 'stages.0.blocks.1', 'stages.0.blocks.1.conv_dw', 'stages.0.blocks.1.norm', 'stages.0.blocks.1.mlp', 'stages.0.blocks.1.mlp.fc1', 'stages.0.blocks.1.mlp.act', 'stages.0.blocks.1.mlp.drop1', 'stages.0.blocks.1.mlp.norm', 'stages.0.blocks.1.mlp.fc2', 'stages.0.blocks.1.mlp.drop2', 'stages.0.blocks.1.shortcut', 'stages.0.blocks.1.drop_path', 'stages.0.blocks.2', 'stages.0.blocks.2.conv_dw', 'stages.0.blocks.2.norm', 'stages.0.blocks.2.mlp', 'stages.0.blocks.2.mlp.fc1', 'stages.0.blocks.2.mlp.act', 'stages.0.blocks.2.mlp.drop1', 'stages.0.blocks.2.ml

# Manage NoisyNN 

## Manually

### Instantiate NoisyNN instances to the model

`n_layers_inject_per_batch`: max number of layers activated NoisyNN in a batch. The real number of layers activated will be `max(len(layers), n_layers_inject_per_batch)`

In [3]:
nnn.inject_noisy_nn(model=model, layers_name= layers, n_layers_inject_per_batch= 1, inplace= True, verbose= True);
# model_new = nnn.inject_noisy_nn(model=model, layers_name= layers, n_layers_inject_per_batch= 1, inplace= False, verbose= True)   <-- In case you don't want to mess with the original model

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->


### Doing what ever you want here with NoisyNN injected model

In [4]:
with nnn.debug_mode(True):   # <-- Just call the debug function, in this case will print the chosen layer.
    model(inp);

Layer stages.3.blocks.2 is chosen!


### Remove NoisyNN instances from the model

In [5]:
nnn.remove_noisy_nn(model=model, inplace= True, verbose= True);
# model_new = nnn.remove_noisy_nn(model=model, inplace= False, verbose= True)  <-- In case you want to keep the NoisyNN injected model?

<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


## Automatically

### As a context manager

In [6]:
with nnn.noisy_nn(model):
    with nnn.debug_mode(enabled= True): # <-- Just call the debug function, in this case will print the chosen layer.
        model(inp);

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->
Layer stages.3.blocks.2 is chosen!
<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


### As a decorator

In [7]:
@nnn.noisy_nn(model)
def infer():
    model(inp);

with nnn.debug_mode(enabled= True): # <-- Just call the debug function, in this case will print the chosen layer.
    infer()

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->
Layer stages.3.blocks.2 is chosen!
<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


# Debugging NoisyNN 


Currently, default debugging only support printing out the chosen layer name. For further customization, see `default_function.py`